<a href="https://colab.research.google.com/github/diegoquintanav/theLMbook/blob/main/RAG_example_hf_and_milvus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://huggingface.co/learn/cookbook/rag_with_hf_and_milvus

In [2]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
! pip install --quiet --upgrade pymilvus sentence-transformers huggingface-hub langchain_community langchain-text-splitters langchain_huggingface pypdf tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.7/345.7 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.3/484.3 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 97.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 116.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.7/437.7 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [4]:
%%bash

if [ ! -f "The-AI-Act.pdf" ]; then
    wget -q https://artificialintelligenceact.eu/wp-content/uploads/2021/08/The-AI-Act.pdf
fi

In [5]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("The-AI-Act.pdf")
docs = loader.load()
print(len(docs))

108


In [29]:
docs[1].to_json()

{'lc': 1,
 'type': 'constructor',
 'id': ['langchain', 'schema', 'document', 'Document'],
 'kwargs': {'metadata': {'producer': 'PDF CoDe 2018 4.7111.7111 (c) 2002-2018 European Commission',
   'creator': 'PDF CoDe 2018 4.7111.7111 (c) 2002-2018 European Commission',
   'creationdate': '2021-04-22T15:27:19+02:00',
   'moddate': '2021-04-22T15:27:19+02:00',
   'title': '',
   'author': '',
   'subject': '',
   'keywords': '',
   'source': 'The-AI-Act.pdf',
   'total_pages': 108,
   'page': 1,
   'page_label': '2'},
  'page_content': 'EN 1  EN \nEXPLANATORY MEMORANDUM \n1. CONTEXT OF THE PROPOSAL \n1.1. Reasons for and objectives of the proposal \nThis explanatory memorandum accompanies the proposal for a Regulation laying down \nharmonised rules on artificial intelligence (Artificial Intelligence Act). Artificial Intelligence \n(AI) is a fast evolving family of technologies that can bring a wide array of economic and \nsocietal benefits across the entire s pectrum of industries and socia

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(docs)

In [8]:
text_lines = [chunk.page_content for chunk in chunks]

In [9]:

text_lines[0]

'EN   EN \n \n \n \nEUROPEAN \nCOMMISSION  \nBrussels, 21.4.2021  \nCOM(2021) 206 final \n2021/0106 (COD) \n \nProposal for a \nREGULATION OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL \nLAYING DOWN HARMONISED RULES ON ARTIFICIAL INTELLIGENCE \n(ARTIFICIAL INTELLIGENCE ACT) AND AMENDING CERTAIN UNION \nLEGISLATIVE ACTS \n{SEC(2021) 167 final} - {SWD(2021) 84 final} - {SWD(2021) 85 final}'

In [10]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("BAAI/bge-small-en-v1.5")


def emb_text(text):
    return embedding_model.encode([text], normalize_embeddings=True).tolist()[0]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
test_embedding = emb_text("This is a test")
embedding_dim = len(test_embedding)
print(embedding_dim)
print(test_embedding[:10])

384
[-0.07660680264234543, 0.02531672641634941, 0.012505538761615753, 0.004595162346959114, 0.02577998675405979, 0.038167111575603485, 0.08050814270973206, 0.0030353872571140528, 0.024392176419496536, 0.004880355205386877]


In [11]:
from pymilvus import MilvusClient

milvus_client = MilvusClient(uri="./hf_milvus_demo.db")

collection_name = "rag_collection"

In [12]:
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

In [15]:
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dim,
    metric_type="IP",  # Inner product distance
    consistency_level="Strong",  # Strong consistency level
)

In [16]:
# insert data
from tqdm import tqdm

data = []

for i, line in enumerate(tqdm(text_lines, desc="Creating embeddings")):
    data.append({"id": i, "vector": emb_text(line), "text": line})

insert_res = milvus_client.insert(collection_name=collection_name, data=data)
insert_res["insert_count"]

Creating embeddings: 100%|██████████| 424/424 [00:07<00:00, 53.33it/s]


424

In [17]:
question = "What is the legal basis for the proposal?"

In [18]:
search_res = milvus_client.search(
    collection_name=collection_name,
    data=[emb_text(question)],  # Use the `emb_text` function to convert the question to an embedding vector
    limit=3,  # Return top 3 results
    search_params={"metric_type": "IP", "params": {}},  # Inner product distance
    output_fields=["text"],  # Return the text field
)

search_res

data: [[{'id': 27, 'distance': 0.7306115031242371, 'entity': {'text': 'EN 6  EN \n2. LEGAL BASIS, SUBSIDIARITY AND PROPORTIONALITY \n2.1. Legal basis \nThe legal basis for the proposal is in the first place Article 114 of the Treaty on the \nFunctioning of the European Union (TFEU), which provides for the adoption of measures to \nensure the establishment and functioning of the internal market.  \nThis proposal constitutes a core part of the EU digital single market strategy. The primary \nobjective of this proposal is to ensure the proper functioning of the internal market by setting \nharmonised rules in particular on the development, placing on the Union market and the use \nof products and services making use of AI technologies or provided as stand -alone AI \nsystems. Some Member States are already considering national rules to ensure that AI is safe \nand is developed and used in compliance with fundamental rights obligations. This will likely \nlead to two main problems: i) a fr

In [19]:
import json

retrieved_lines_with_distances = [(res["entity"]["text"], res["distance"]) for res in search_res[0]]
print(json.dumps(retrieved_lines_with_distances, indent=4))

[
    [
        "EN 6  EN \n2. LEGAL BASIS, SUBSIDIARITY AND PROPORTIONALITY \n2.1. Legal basis \nThe legal basis for the proposal is in the first place Article 114 of the Treaty on the \nFunctioning of the European Union (TFEU), which provides for the adoption of measures to \nensure the establishment and functioning of the internal market.  \nThis proposal constitutes a core part of the EU digital single market strategy. The primary \nobjective of this proposal is to ensure the proper functioning of the internal market by setting \nharmonised rules in particular on the development, placing on the Union market and the use \nof products and services making use of AI technologies or provided as stand -alone AI \nsystems. Some Member States are already considering national rules to ensure that AI is safe \nand is developed and used in compliance with fundamental rights obligations. This will likely \nlead to two main problems: i) a fragmentation of the internal market on essential elemen

In [20]:
context = "\n".join([line_with_distance[0] for line_with_distance in retrieved_lines_with_distances])

In [21]:
PROMPT = """
Use the following pieces of information enclosed in <context> tags to provide an answer to the question enclosed in <question> tags.
<context>
{context}
</context>
<question>
{question}
</question>
"""

In [22]:
from huggingface_hub import InferenceClient

repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

llm_client = InferenceClient(model=repo_id, timeout=120)

In [23]:
prompt = PROMPT.format(context=context, question=question)

In [24]:
answer = llm_client.text_generation(
    prompt,
    max_new_tokens=1000,
).strip()
print(answer)

The legal basis for the proposal is Article 114 of the Treaty on the Functioning of the European Union (TFEU), which provides for the adoption of measures to ensure the establishment and functioning of the internal market.


In [26]:
llm_client.text_generation('hablas castellano?')

"\n\nI'm not sure if this is the right place to post this, but I'm looking for a Spanish language tutor. I'm a beginner, and I'm looking for someone who can help me learn the language. I'm willing to pay for lessons, but I'm not sure how much they should cost. Can anyone recommend a good tutor or give me an idea of how much I should expect to pay? Thanks in advance for your help!\n\nI'm not sure where you're located, but you might want to try looking for a tutor on a website like Wyzant or Tutor.com. These websites allow you to search for tutors in your area and read reviews from other students. You can also specify that you're looking for a tutor who speaks Spanish, which should help you find someone who can help you with the language.\n\nAs for how much you should expect to pay, it really depends on the tutor's qualifications and experience. In general, you can expect to pay anywhere from $20 to $60 per hour for a Spanish language tutor. However, some tutors may charge more or less t

In [30]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [27]:
# Use a pipeline as a high-level helper
from transformers import pipeline

messages = [
    {"role": "user", "content": "Hablas castellano?"},
]
pipe = pipeline("text-generation", model="google/gemma-3-1b-it")
pipe(messages)

config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

Device set to use cuda:0


[{'generated_text': [{'role': 'user', 'content': 'Hablas castellano?'},
   {'role': 'assistant',
    'content': '¡Sí, claro que hablo castellano! ¿En qué puedo ayudarte?\n'}]}]

In [36]:
from pathlib import Path
JSON_DATA = (Path("/")
    / "content"
    / "drive"
    / "MyDrive"
    / "Almudata"
    / "03-centre-ajuda-rag" /"data"/"preprocessing"/"json"
)

INDEX_JSON_PATH = JSON_DATA / "_index.json"

In [37]:
INDEX_JSON_PATH

PosixPath('/content/drive/MyDrive/Almudata/03-centre-ajuda-rag/data/preprocessing/json/_index.json')

In [38]:
list(INDEX_JSON_PATH.parent.glob('*.json'))[:5]

[PosixPath('/content/drive/MyDrive/Almudata/03-centre-ajuda-rag/data/preprocessing/json/21da_criterios.json'),
 PosixPath('/content/drive/MyDrive/Almudata/03-centre-ajuda-rag/data/preprocessing/json/093c_que-es-gur.json'),
 PosixPath('/content/drive/MyDrive/Almudata/03-centre-ajuda-rag/data/preprocessing/json/0355_si-estoy-d.json'),
 PosixPath('/content/drive/MyDrive/Almudata/03-centre-ajuda-rag/data/preprocessing/json/8cb9_puedo-acc.json'),
 PosixPath('/content/drive/MyDrive/Almudata/03-centre-ajuda-rag/data/preprocessing/json/45c0_por-que-e.json')]

In [39]:
print(list(INDEX_JSON_PATH.parent.glob('*.json'))[0].read_text())

{
  "raw_heading": "<h3>Criterios para adherirse a GURB</h3>",
  "heading": "Criterios para adherirse a GURB",
  "heading_as_question": "¿Criterios para adherirse a GURB?",
  "raw_paragraph": "<p>De forma resumida, para que te puedas adherir a GURB debes cumplir los siguientes criterios:</p> <ul>\n<li>Disponer de un contrato de suministro eléctrico activo en Som Energia que se encuentre ubicado en un radio de <strong>2.000 m en torno a la ubicación de la instalación fotovoltaica productora.</strong></li>\n<li>No tener ninguna tipología de autoconsumo activa en tu contrato de suministro.</li>\n</ul> <li>Disponer de un contrato de suministro eléctrico activo en Som Energia que se encuentre ubicado en un radio de <strong>2.000 m en torno a la ubicación de la instalación fotovoltaica productora.</strong></li> <li>No tener ninguna tipología de autoconsumo activa en tu contrato de suministro.</li> <p>Al mismo tiempo, debes tener en cuenta que, por motivos técnicos, cuando pidas añadirte a un

In [31]:
from pymilvus import MilvusClient

milvus_client = MilvusClient(uri="./hf_som_es_milvus.db")

collection_name = "som_es_rag_collection"

In [32]:
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

In [33]:
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dim,
    metric_type="IP",  # Inner product distance
    consistency_level="Strong",  # Strong consistency level
)

In [ ]:
import json

In [54]:
_text_lines_json = [json.load(p.open('rt')) for p in INDEX_JSON_PATH.parent.glob('*.json')]



In [75]:
_text_lines_json[0]

{'raw_heading': '<h3>Criterios para adherirse a GURB</h3>',
 'heading': 'Criterios para adherirse a GURB',
 'heading_as_question': '¿Criterios para adherirse a GURB?',
 'raw_paragraph': '<p>De forma resumida, para que te puedas adherir a GURB debes cumplir los siguientes criterios:</p> <ul>\n<li>Disponer de un contrato de suministro eléctrico activo en Som Energia que se encuentre ubicado en un radio de <strong>2.000 m en torno a la ubicación de la instalación fotovoltaica productora.</strong></li>\n<li>No tener ninguna tipología de autoconsumo activa en tu contrato de suministro.</li>\n</ul> <li>Disponer de un contrato de suministro eléctrico activo en Som Energia que se encuentre ubicado en un radio de <strong>2.000 m en torno a la ubicación de la instalación fotovoltaica productora.</strong></li> <li>No tener ninguna tipología de autoconsumo activa en tu contrato de suministro.</li> <p>Al mismo tiempo, debes tener en cuenta que, por motivos técnicos, cuando pidas añadirte a un proye

In [57]:
type(_text_lines_json[0])

dict

In [86]:
text_lines = []
for x in _text_lines_json:
  try:
    _line = x['heading'] + ': ' + x['paragraph']
    text_lines.append(_line)
  except Exception as e:
    print(e)
    print(type(_line))
    print(x)
    print(_line)
    continue


list indices must be integers or slices, not str
<class 'str'>
[{'title': '¿Cómo puedo saber mi número de socio/a?', 'url': 'https://es.support.somenergia.coop/article/865-como-puedo-saber-mi-numero-de-socio-a', 'category': '138-todavia-no-tengo-la-luz-contratada', 'url_slug': '865-como-puedo-saber-mi-numero-de-socio-a', 'md_path': '/home/blmnk/Code/me/somenergia-centre-ajuda-agent-project/somenergia-centre-ajuda-scraper/out/scraping/2025-04-25/es.support.somenergia.coop/articles/138-todavia-no-tengo-la-luz-contratada/md/865-como-puedo-saber-mi-numero-de-socio-a.md', 'json_path': '/home/blmnk/Code/me/somenergia-centre-ajuda-agent-project/somenergia-centre-ajuda-scraper/out/scraping/2025-04-25/es.support.somenergia.coop/articles/138-todavia-no-tengo-la-luz-contratada/json/865-como-puedo-saber-mi-numero-de-socio-a.json', 'html_path': '/home/blmnk/Code/me/somenergia-centre-ajuda-agent-project/somenergia-centre-ajuda-scraper/out/scraping/2025-04-25/es.support.somenergia.coop/articles/138-t

In [87]:
text_lines[0]

'Criterios para adherirse a GURB: De forma resumida para que te puedas adherir a GURB debes cumplir los siguientes criterios Disponer de un contrato de suministro eléctrico activo en Som Energia que se encuentre ubicado en un radio de 2000 m en torno a la ubicación de la instalación fotovoltaica productora No tener ninguna tipología de autoconsumo activa en tu contrato de suministro Disponer de un contrato de suministro eléctrico activo en Som Energia que se encuentre ubicado en un radio de 2000 m en torno a la ubicación de la instalación fotovoltaica productora No tener ninguna tipología de autoconsumo activa en tu contrato de suministro Al mismo tiempo debes tener en cuenta que por motivos técnicos cuando pidas añadirte a un proyecto GURB no puedes tener en curso ninguna modificación de potencia tensión o tarifa en trámite En este caso tienes que esperar a que esta modificación finalice para poder adherirte a GURB'

In [88]:
# insert data
from tqdm import tqdm

data = []

for i, line in enumerate(tqdm(text_lines, desc="Creating embeddings")):
    data.append({"id": i, "vector": emb_text(line), "text": line})

insert_res = milvus_client.insert(collection_name=collection_name, data=data)
insert_res["insert_count"]

Creating embeddings: 100%|██████████| 1028/1028 [00:15<00:00, 65.98it/s]


1028

In [89]:
question = "¿Cómo me adhiero a GURB?"

In [90]:
search_res = milvus_client.search(
    collection_name=collection_name,
    data=[emb_text(question)],  # Use the `emb_text` function to convert the question to an embedding vector
    limit=3,  # Return top 3 results
    search_params={"metric_type": "IP", "params": {}},  # Inner product distance
    output_fields=["text"],  # Return the text field
)

search_res

data: [[{'id': 7, 'distance': 0.7816780805587769, 'entity': {'text': '¿Qué potencia GURB puedo contratar?: Tal y como explicamos en este artículo GURB generación eléctrica urbana es el nuevo servicio de Som Energia que facilita la participación en un proyecto de autoconsumo colectivo a las personas y entidades socias de la cooperativa especialmente aquellas que por las circunstancias que sea no pueden participar de un autoconsumo En este artículo hablaremos de los siguientes puntos Qué es la potencia GURB Qué potencia GURB puedo contratar De acuerdo con mi consumo cuál es mi potencia GURB recomendada Qué es la potencia GURB Qué potencia GURB puedo contratar De acuerdo con mi consumo cuál es mi potencia GURB recomendada'}}, {'id': 520, 'distance': 0.7804588079452515, 'entity': {'text': '¿Qué potencia GURB puedo contratar?: Tal y como explicamos en este artículo, GURB (generación eléctrica urbana) es el nuevo servicio de Som Energia que facilita la participación en un proyecto de autocon

In [91]:
import json

retrieved_lines_with_distances = [(res["entity"]["text"], res["distance"]) for res in search_res[0]]
print(json.dumps(retrieved_lines_with_distances, indent=4))

[
    [
        "\u00bfQu\u00e9 potencia GURB puedo contratar?: Tal y como explicamos en este art\u00edculo GURB generaci\u00f3n el\u00e9ctrica urbana es el nuevo servicio de Som Energia que facilita la participaci\u00f3n en un proyecto de autoconsumo colectivo a las personas y entidades socias de la cooperativa especialmente aquellas que por las circunstancias que sea no pueden participar de un autoconsumo En este art\u00edculo hablaremos de los siguientes puntos Qu\u00e9 es la potencia GURB Qu\u00e9 potencia GURB puedo contratar De acuerdo con mi consumo cu\u00e1l es mi potencia GURB recomendada Qu\u00e9 es la potencia GURB Qu\u00e9 potencia GURB puedo contratar De acuerdo con mi consumo cu\u00e1l es mi potencia GURB recomendada",
        0.7816780805587769
    ],
    [
        "\u00bfQu\u00e9 potencia GURB puedo contratar?: Tal y como explicamos en este art\u00edculo, GURB (generaci\u00f3n el\u00e9ctrica urbana) es el nuevo servicio de Som Energia que facilita la participaci\u00f3n 

In [92]:
context = "\n".join([line_with_distance[0] for line_with_distance in retrieved_lines_with_distances])

In [93]:
PROMPT = """
Utiliza las siguientes piezas de información encrutadas en <context> tags para proporcionar una respuesta a la pregunta encrutada en <question> tags.
<context>
{context}
</context>
<question>
{question}
</question>
"""

In [95]:
# Use a pipeline as a high-level helper
from transformers import pipeline

messages = [
    {"role": "user", "content": PROMPT.format(context=context, question=question)},
]
pipe = pipeline("text-generation", model="google/gemma-3-1b-it", max_new_tokens=1000)
pipe(messages)

Device set to use cuda:0


[{'generated_text': [{'role': 'user',
    'content': '\nUtiliza las siguientes piezas de información encrutadas en <context> tags para proporcionar una respuesta a la pregunta encrutada en <question> tags.\n<context>\n¿Qué potencia GURB puedo contratar?: Tal y como explicamos en este artículo GURB generación eléctrica urbana es el nuevo servicio de Som Energia que facilita la participación en un proyecto de autoconsumo colectivo a las personas y entidades socias de la cooperativa especialmente aquellas que por las circunstancias que sea no pueden participar de un autoconsumo En este artículo hablaremos de los siguientes puntos Qué es la potencia GURB Qué potencia GURB puedo contratar De acuerdo con mi consumo cuál es mi potencia GURB recomendada Qué es la potencia GURB Qué potencia GURB puedo contratar De acuerdo con mi consumo cuál es mi potencia GURB recomendada\n¿Qué potencia GURB puedo contratar?: Tal y como explicamos en este artículo, GURB (generación eléctrica urbana) es el nuev